In [1]:
import pandas as pd
import numpy as np

In [2]:
allocation_csvs = ["ccl", "expansion", "expansion_draft", "leagues_cup","playoffs"]

base_additional_allocations = pd.DataFrame()
for item in allocation_csvs:
    file_path = f"./allocation_money/{item}_allocations.csv"
    tmp = pd.read_csv(file_path)
    base_additional_allocations = base_additional_allocations.append(tmp)
base_additional_allocations

,season,club,allocation_condition,gam_value
0,2018,Toronto FC,CCL,200000
1,2018,Kansas City,CCL,200000
2,2018,FC Dallas,CCL,200000
3,2018,Colorado Rapids,CCL,200000
4,2018,Seattle Sounders,CCL,200000
...,...,...,...,...
48,2022,LAFC,No Playoffs,200000
49,2022,San Jose,No Playoffs,200000
50,2022,FC Dallas,No Playoffs,200000
51,2022,Austin FC,No Playoffs,200000


In [3]:
asset_expl_flat = pd.read_csv('./allocation_money/mls_trade_db.csv')
asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year,tam_value
0,0,2020-12-29,New York City FC,Player,$250000 2021 GAM,0,250000,2021,0
1,0,2020-12-29,New York City FC,Player,$250000 2022 GAM,1,250000,2022,0
2,0,2020-12-29,New York City FC,Player,$100000 2022 GAM (performance bonus),2,100000,2022,0
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020,0
4,1,2020-12-23,New York Red Bulls,Player,$50000 2021 GAM,4,50000,2021,0
...,...,...,...,...,...,...,...,...,...
522,190,2019-01-11,Houston,Draft Pick,2019 2nd round pick (#33),522,0,2019,0
523,190,2019-01-11,D.C. United,Player,Chris Seitz (GK),523,0,2019,0
524,191,2019-01-11,Houston,Player,Marlon Hairston,524,0,2019,0
525,191,2019-01-11,Colorado Rapids,Player,$125000 2019 GAM,525,125000,2019,0


In [4]:
asset_expl_flat.acquired_asset_type = np.select([
    asset_expl_flat.acquired_asset_value.str.contains('GAM'),
    asset_expl_flat.acquired_asset_value.str.contains('allocation')
],[
    'GAM',
    'GAM'
],
default=asset_expl_flat.acquired_asset_type)

In [5]:
list(set(base_additional_allocations.club.unique()) - set(asset_expl_flat.club.unique()))

['Minnesota United', 'Portland Timbers', 'Colorado']

In [6]:
import numpy as np
base_additional_allocations.club = np.select(
    [
        (base_additional_allocations.club == 'Colorado'),
        (base_additional_allocations.club == 'Minnesota United'),
        (base_additional_allocations.club == 'Portland Timbers')
    ],
    [
        'Colorado Rapids',
        'Minnesota',
        'Portland'
    ]
    ,default = base_additional_allocations.club)
base_additional_allocations

,season,club,allocation_condition,gam_value
0,2018,Toronto FC,CCL,200000
1,2018,Kansas City,CCL,200000
2,2018,FC Dallas,CCL,200000
3,2018,Colorado Rapids,CCL,200000
4,2018,Seattle Sounders,CCL,200000
...,...,...,...,...
48,2022,LAFC,No Playoffs,200000
49,2022,San Jose,No Playoffs,200000
50,2022,FC Dallas,No Playoffs,200000
51,2022,Austin FC,No Playoffs,200000


In [7]:
list(set(base_additional_allocations.club.unique()) - set(asset_expl_flat.club.unique()))

[]

In [8]:
list(set(base_additional_allocations.columns) - set(asset_expl_flat.columns))

['allocation_condition', 'season']

In [9]:
list(set(asset_expl_flat.columns) - set(base_additional_allocations.columns))

['acquired_asset_value',
 'trade_id',
 'asset_year',
 'tam_value',
 'trade_date',
 'acquired_asset_type',
 'acquired_asset_id']

In [10]:
# import dt
base_additional_allocations['raw_trade_date'] = base_additional_allocations.agg('1/1/{0[season]}'.format, axis=1)
base_additional_allocations['trade_date'] = pd.to_datetime(base_additional_allocations['raw_trade_date'], errors='coerce')
base_additional_allocations['acquired_asset_value'] = base_additional_allocations.agg('${0[gam_value]} {0[season]} {0[allocation_condition]} allocation'.format, axis=1)
base_additional_allocations['acquired_asset_type'] = 'GAM'
base_additional_allocations['asset_year'] = base_additional_allocations['season']
base_additional_allocations['tam_value'] = 0
base_additional_allocations

,season,club,allocation_condition,gam_value,raw_trade_date,trade_date,acquired_asset_value,acquired_asset_type,asset_year,tam_value
0,2018,Toronto FC,CCL,200000,1/1/2018,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
1,2018,Kansas City,CCL,200000,1/1/2018,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
2,2018,FC Dallas,CCL,200000,1/1/2018,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
3,2018,Colorado Rapids,CCL,200000,1/1/2018,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
4,2018,Seattle Sounders,CCL,200000,1/1/2018,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
...,...,...,...,...,...,...,...,...,...,...
48,2022,LAFC,No Playoffs,200000,1/1/2022,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0
49,2022,San Jose,No Playoffs,200000,1/1/2022,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0
50,2022,FC Dallas,No Playoffs,200000,1/1/2022,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0
51,2022,Austin FC,No Playoffs,200000,1/1/2022,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0


In [11]:
adj_allocations = base_additional_allocations.drop(['allocation_condition', 'raw_trade_date','season'], axis=1)
adj_allocations

,club,gam_value,trade_date,acquired_asset_value,acquired_asset_type,asset_year,tam_value
0,Toronto FC,200000,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
1,Kansas City,200000,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
2,FC Dallas,200000,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
3,Colorado Rapids,200000,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
4,Seattle Sounders,200000,2018-01-01,$200000 2018 CCL allocation,GAM,2018,0
...,...,...,...,...,...,...,...
48,LAFC,200000,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0
49,San Jose,200000,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0
50,FC Dallas,200000,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0
51,Austin FC,200000,2022-01-01,$200000 2022 No Playoffs allocation,GAM,2022,0


In [12]:
base_len = len(asset_expl_flat)
base_id = asset_expl_flat.trade_id.max() + 1

adj_allocations['trade_id'] = range(base_id, base_id + len(adj_allocations))
adj_allocations['acquired_asset_id'] = range(base_len, base_len + len(adj_allocations))
adj_allocations = adj_allocations[['trade_id','trade_date', 'club', 'acquired_asset_type','acquired_asset_value','acquired_asset_id','gam_value','asset_year','tam_value']]
adj_allocations

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year,tam_value
0,192,2018-01-01,Toronto FC,GAM,$200000 2018 CCL allocation,527,200000,2018,0
1,193,2018-01-01,Kansas City,GAM,$200000 2018 CCL allocation,528,200000,2018,0
2,194,2018-01-01,FC Dallas,GAM,$200000 2018 CCL allocation,529,200000,2018,0
3,195,2018-01-01,Colorado Rapids,GAM,$200000 2018 CCL allocation,530,200000,2018,0
4,196,2018-01-01,Seattle Sounders,GAM,$200000 2018 CCL allocation,531,200000,2018,0
...,...,...,...,...,...,...,...,...,...
48,316,2022-01-01,LAFC,GAM,$200000 2022 No Playoffs allocation,651,200000,2022,0
49,317,2022-01-01,San Jose,GAM,$200000 2022 No Playoffs allocation,652,200000,2022,0
50,318,2022-01-01,FC Dallas,GAM,$200000 2022 No Playoffs allocation,653,200000,2022,0
51,319,2022-01-01,Austin FC,GAM,$200000 2022 No Playoffs allocation,654,200000,2022,0


In [13]:
composite_allocations = asset_expl_flat.append(adj_allocations)
composite_allocations

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year,tam_value
0,0,2020-12-29,New York City FC,GAM,$250000 2021 GAM,0,250000,2021,0
1,0,2020-12-29,New York City FC,GAM,$250000 2022 GAM,1,250000,2022,0
2,0,2020-12-29,New York City FC,GAM,$100000 2022 GAM (performance bonus),2,100000,2022,0
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020,0
4,1,2020-12-23,New York Red Bulls,GAM,$50000 2021 GAM,4,50000,2021,0
...,...,...,...,...,...,...,...,...,...
48,316,2022-01-01 00:00:00,LAFC,GAM,$200000 2022 No Playoffs allocation,651,200000,2022,0
49,317,2022-01-01 00:00:00,San Jose,GAM,$200000 2022 No Playoffs allocation,652,200000,2022,0
50,318,2022-01-01 00:00:00,FC Dallas,GAM,$200000 2022 No Playoffs allocation,653,200000,2022,0
51,319,2022-01-01 00:00:00,Austin FC,GAM,$200000 2022 No Playoffs allocation,654,200000,2022,0


In [14]:
composite_allocations.acquired_asset_id.nunique()

656

In [15]:
composite_allocations.columns

Index(['trade_id', 'trade_date', 'club', 'acquired_asset_type',
       'acquired_asset_value', 'acquired_asset_id', 'gam_value', 'asset_year',
       'tam_value'],
      dtype='object')

In [16]:
composite_allocations.columns = ['transaction_id', 'transaction_date', 'club', 'acquired_asset_type',
       'acquired_asset_value', 'acquired_asset_id', 'gam_value', 'asset_year',
       'tam_value']
composite_allocations

,transaction_id,transaction_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year,tam_value
0,0,2020-12-29,New York City FC,GAM,$250000 2021 GAM,0,250000,2021,0
1,0,2020-12-29,New York City FC,GAM,$250000 2022 GAM,1,250000,2022,0
2,0,2020-12-29,New York City FC,GAM,$100000 2022 GAM (performance bonus),2,100000,2022,0
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020,0
4,1,2020-12-23,New York Red Bulls,GAM,$50000 2021 GAM,4,50000,2021,0
...,...,...,...,...,...,...,...,...,...
48,316,2022-01-01 00:00:00,LAFC,GAM,$200000 2022 No Playoffs allocation,651,200000,2022,0
49,317,2022-01-01 00:00:00,San Jose,GAM,$200000 2022 No Playoffs allocation,652,200000,2022,0
50,318,2022-01-01 00:00:00,FC Dallas,GAM,$200000 2022 No Playoffs allocation,653,200000,2022,0
51,319,2022-01-01 00:00:00,Austin FC,GAM,$200000 2022 No Playoffs allocation,654,200000,2022,0


In [17]:
composite_allocations.to_csv('./allocation_money/mls_transactions_db.csv', index=False)

In [18]:
am_table = {  }

available_asset_years = range(int(asset_expl_flat.asset_year.min()), int(asset_expl_flat.asset_year.max()) + 1)
for year in available_asset_years:
    am_table[str(year)] = {}
    for club in asset_expl_flat.club.unique():
        am_table[str(year)][club] = {}
        am_table[str(year)][club]['GAM'] = 0
        am_table[str(year)][club]['TAM'] = 0
        

    
# there's a three team trade in here to look out for, ignore it
grouped_trades = asset_expl_flat.groupby('trade_id')
for trade_id, trade in grouped_trades:
    gam_groups = trade.groupby('club')

    if (len(gam_groups) == 2):
        for club, rows in gam_groups:
            for year in available_asset_years:
                selected_asset_year = str(year)
                used_assets = rows.acquired_asset_id.tolist()
                
                gam_add = rows[rows.asset_year == year].gam_value.sum()
                gam_remove = trade[(trade.asset_year == year) & ~(trade.acquired_asset_id.isin(used_assets))].gam_value.sum()
                
                print(f'{selected_asset_year}: Adding {gam_add} GAM to {club}')
                am_table[selected_asset_year][club]['GAM'] += gam_add
                print(f'{selected_asset_year}: Subtracting {gam_remove} GAM from {club}')
                am_table[selected_asset_year][club]['GAM'] -= gam_remove
                
                tam_add = rows[rows.asset_year == year].tam_value.sum()
                tam_remove = trade[(trade.asset_year == year) & ~(trade.acquired_asset_id.isin(used_assets))].tam_value.sum()
                
                print(f'{selected_asset_year}: Adding {tam_add} TAM to {club}')
                am_table[selected_asset_year][club]['TAM'] += tam_add
                print(f'{selected_asset_year}: Subtracting {tam_remove} TAM from {club}')
                am_table[selected_asset_year][club]['TAM'] -= tam_remove

    else:
        print(f'trade_id {trade_id} had weird number of teams: {len(gam_groups)}')
        
am_table  

2018: Adding 0 GAM to FC Cincinnati
2018: Subtracting 0 GAM from FC Cincinnati
2018: Adding 0 TAM to FC Cincinnati
2018: Subtracting 0 TAM from FC Cincinnati
2019: Adding 0 GAM to FC Cincinnati
2019: Subtracting 0 GAM from FC Cincinnati
2019: Adding 0 TAM to FC Cincinnati
2019: Subtracting 0 TAM from FC Cincinnati
2020: Adding 0 GAM to FC Cincinnati
2020: Subtracting 0 GAM from FC Cincinnati
2020: Adding 0 TAM to FC Cincinnati
2020: Subtracting 0 TAM from FC Cincinnati
2021: Adding 0 GAM to FC Cincinnati
2021: Subtracting 250000 GAM from FC Cincinnati
2021: Adding 0 TAM to FC Cincinnati
2021: Subtracting 0 TAM from FC Cincinnati
2022: Adding 0 GAM to FC Cincinnati
2022: Subtracting 350000 GAM from FC Cincinnati
2022: Adding 0 TAM to FC Cincinnati
2022: Subtracting 0 TAM from FC Cincinnati
2023: Adding 0 GAM to FC Cincinnati
2023: Subtracting 0 GAM from FC Cincinnati
2023: Adding 0 TAM to FC Cincinnati
2023: Subtracting 0 TAM from FC Cincinnati
2024: Adding 0 GAM to FC Cincinnati
2024: 

2024: Adding 0 GAM to Austin FC
2024: Subtracting 0 GAM from Austin FC
2024: Adding 0 TAM to Austin FC
2024: Subtracting 0 TAM from Austin FC
2025: Adding 0 GAM to Austin FC
2025: Subtracting 0 GAM from Austin FC
2025: Adding 0 TAM to Austin FC
2025: Subtracting 0 TAM from Austin FC
2018: Adding 0 GAM to Inter Miami
2018: Subtracting 0 GAM from Inter Miami
2018: Adding 0 TAM to Inter Miami
2018: Subtracting 0 TAM from Inter Miami
2019: Adding 0 GAM to Inter Miami
2019: Subtracting 0 GAM from Inter Miami
2019: Adding 0 TAM to Inter Miami
2019: Subtracting 0 TAM from Inter Miami
2020: Adding 0 GAM to Inter Miami
2020: Subtracting 0 GAM from Inter Miami
2020: Adding 0 TAM to Inter Miami
2020: Subtracting 0 TAM from Inter Miami
2021: Adding 100000 GAM to Inter Miami
2021: Subtracting 0 GAM from Inter Miami
2021: Adding 0 TAM to Inter Miami
2021: Subtracting 0 TAM from Inter Miami
2022: Adding 0 GAM to Inter Miami
2022: Subtracting 0 GAM from Inter Miami
2022: Adding 0 TAM to Inter Miami
20

2018: Adding 0 GAM to Inter Miami
2018: Subtracting 0 GAM from Inter Miami
2018: Adding 0 TAM to Inter Miami
2018: Subtracting 0 TAM from Inter Miami
2019: Adding 0 GAM to Inter Miami
2019: Subtracting 0 GAM from Inter Miami
2019: Adding 0 TAM to Inter Miami
2019: Subtracting 0 TAM from Inter Miami
2020: Adding 0 GAM to Inter Miami
2020: Subtracting 0 GAM from Inter Miami
2020: Adding 0 TAM to Inter Miami
2020: Subtracting 0 TAM from Inter Miami
2021: Adding 0 GAM to Inter Miami
2021: Subtracting 0 GAM from Inter Miami
2021: Adding 0 TAM to Inter Miami
2021: Subtracting 0 TAM from Inter Miami
2022: Adding 0 GAM to Inter Miami
2022: Subtracting 0 GAM from Inter Miami
2022: Adding 0 TAM to Inter Miami
2022: Subtracting 0 TAM from Inter Miami
2023: Adding 0 GAM to Inter Miami
2023: Subtracting 0 GAM from Inter Miami
2023: Adding 0 TAM to Inter Miami
2023: Subtracting 0 TAM from Inter Miami
2024: Adding 0 GAM to Inter Miami
2024: Subtracting 0 GAM from Inter Miami
2024: Adding 0 TAM to Int

2019: Adding 0 TAM to New England
2019: Subtracting 0 TAM from New England
2020: Adding 0 GAM to New England
2020: Subtracting 0 GAM from New England
2020: Adding 0 TAM to New England
2020: Subtracting 0 TAM from New England
2021: Adding 0 GAM to New England
2021: Subtracting 0 GAM from New England
2021: Adding 0 TAM to New England
2021: Subtracting 0 TAM from New England
2022: Adding 0 GAM to New England
2022: Subtracting 0 GAM from New England
2022: Adding 0 TAM to New England
2022: Subtracting 0 TAM from New England
2023: Adding 0 GAM to New England
2023: Subtracting 0 GAM from New England
2023: Adding 0 TAM to New England
2023: Subtracting 0 TAM from New England
2024: Adding 0 GAM to New England
2024: Subtracting 0 GAM from New England
2024: Adding 0 TAM to New England
2024: Subtracting 0 TAM from New England
2025: Adding 0 GAM to New England
2025: Subtracting 0 GAM from New England
2025: Adding 0 TAM to New England
2025: Subtracting 0 TAM from New England
2018: Adding 0 GAM to Hou

2024: Adding 0 GAM to Nashville
2024: Subtracting 0 GAM from Nashville
2024: Adding 0 TAM to Nashville
2024: Subtracting 0 TAM from Nashville
2025: Adding 0 GAM to Nashville
2025: Subtracting 0 GAM from Nashville
2025: Adding 0 TAM to Nashville
2025: Subtracting 0 TAM from Nashville
2018: Adding 0 GAM to Inter Miami
2018: Subtracting 0 GAM from Inter Miami
2018: Adding 0 TAM to Inter Miami
2018: Subtracting 0 TAM from Inter Miami
2019: Adding 0 GAM to Inter Miami
2019: Subtracting 0 GAM from Inter Miami
2019: Adding 0 TAM to Inter Miami
2019: Subtracting 0 TAM from Inter Miami
2020: Adding 0 GAM to Inter Miami
2020: Subtracting 0 GAM from Inter Miami
2020: Adding 0 TAM to Inter Miami
2020: Subtracting 0 TAM from Inter Miami
2021: Adding 0 GAM to Inter Miami
2021: Subtracting 0 GAM from Inter Miami
2021: Adding 0 TAM to Inter Miami
2021: Subtracting 0 TAM from Inter Miami
2022: Adding 0 GAM to Inter Miami
2022: Subtracting 0 GAM from Inter Miami
2022: Adding 0 TAM to Inter Miami
2022: S

...

2025: Adding 0 GAM to Nashville
2025: Subtracting 0 GAM from Nashville
2025: Adding 0 TAM to Nashville
2025: Subtracting 0 TAM from Nashville
2018: Adding 0 GAM to LA Galaxy
2018: Subtracting 0 GAM from LA Galaxy
2018: Adding 0 TAM to LA Galaxy
2018: Subtracting 0 TAM from LA Galaxy
2019: Adding 0 GAM to LA Galaxy
2019: Subtracting 0 GAM from LA Galaxy
2019: Adding 0 TAM to LA Galaxy
2019: Subtracting 0 TAM from LA Galaxy
2020: Adding 225000 GAM to LA Galaxy
2020: Subtracting 0 GAM from LA Galaxy
2020: Adding 0 TAM to LA Galaxy
2020: Subtracting 0 TAM from LA Galaxy
2021: Adding 0 GAM to LA Galaxy
2021: Subtracting 0 GAM from LA Galaxy
2021: Adding 0 TAM to LA Galaxy
2021: Subtracting 0 TAM from LA Galaxy
2022: Adding 0 GAM to LA Galaxy
2022: Subtracting 0 GAM from LA Galaxy
2022: Adding 0 TAM to LA Galaxy
2022: Subtracting 0 TAM from LA Galaxy
2023: Adding 0 GAM to LA Galaxy
2023: Subtracting 0 GAM from LA Galaxy
2023: Adding 0 TAM to LA Galaxy
2023: Subtracting 0 TAM from LA Galaxy
2

2018: Adding 0 TAM to Minnesota
2018: Subtracting 0 TAM from Minnesota
2019: Adding 50000 GAM to Minnesota
2019: Subtracting 0 GAM from Minnesota
2019: Adding 0 TAM to Minnesota
2019: Subtracting 0 TAM from Minnesota
2020: Adding 0 GAM to Minnesota
2020: Subtracting 0 GAM from Minnesota
2020: Adding 0 TAM to Minnesota
2020: Subtracting 0 TAM from Minnesota
2021: Adding 0 GAM to Minnesota
2021: Subtracting 0 GAM from Minnesota
2021: Adding 0 TAM to Minnesota
2021: Subtracting 0 TAM from Minnesota
2022: Adding 0 GAM to Minnesota
2022: Subtracting 0 GAM from Minnesota
2022: Adding 0 TAM to Minnesota
2022: Subtracting 0 TAM from Minnesota
2023: Adding 0 GAM to Minnesota
2023: Subtracting 0 GAM from Minnesota
2023: Adding 0 TAM to Minnesota
2023: Subtracting 0 TAM from Minnesota
2024: Adding 0 GAM to Minnesota
2024: Subtracting 0 GAM from Minnesota
2024: Adding 0 TAM to Minnesota
2024: Subtracting 0 TAM from Minnesota
2025: Adding 0 GAM to Minnesota
2025: Subtracting 0 GAM from Minnesota
20

2018: Adding 0 GAM to Colorado Rapids
2018: Subtracting 0 GAM from Colorado Rapids
2018: Adding 0 TAM to Colorado Rapids
2018: Subtracting 0 TAM from Colorado Rapids
2019: Adding 0 GAM to Colorado Rapids
2019: Subtracting 0 GAM from Colorado Rapids
2019: Adding 50000 TAM to Colorado Rapids
2019: Subtracting 0 TAM from Colorado Rapids
2020: Adding 0 GAM to Colorado Rapids
2020: Subtracting 0 GAM from Colorado Rapids
2020: Adding 0 TAM to Colorado Rapids
2020: Subtracting 0 TAM from Colorado Rapids
2021: Adding 0 GAM to Colorado Rapids
2021: Subtracting 0 GAM from Colorado Rapids
2021: Adding 0 TAM to Colorado Rapids
2021: Subtracting 0 TAM from Colorado Rapids
2022: Adding 0 GAM to Colorado Rapids
2022: Subtracting 0 GAM from Colorado Rapids
2022: Adding 0 TAM to Colorado Rapids
2022: Subtracting 0 TAM from Colorado Rapids
2023: Adding 0 GAM to Colorado Rapids
2023: Subtracting 0 GAM from Colorado Rapids
2023: Adding 0 TAM to Colorado Rapids
2023: Subtracting 0 TAM from Colorado Rapids


2018: Subtracting 0 TAM from Kansas City
2019: Adding 900000 GAM to Kansas City
2019: Subtracting 0 GAM from Kansas City
2019: Adding 0 TAM to Kansas City
2019: Subtracting 0 TAM from Kansas City
2020: Adding 0 GAM to Kansas City
2020: Subtracting 0 GAM from Kansas City
2020: Adding 0 TAM to Kansas City
2020: Subtracting 0 TAM from Kansas City
2021: Adding 0 GAM to Kansas City
2021: Subtracting 0 GAM from Kansas City
2021: Adding 0 TAM to Kansas City
2021: Subtracting 0 TAM from Kansas City
2022: Adding 0 GAM to Kansas City
2022: Subtracting 0 GAM from Kansas City
2022: Adding 0 TAM to Kansas City
2022: Subtracting 0 TAM from Kansas City
2023: Adding 0 GAM to Kansas City
2023: Subtracting 0 GAM from Kansas City
2023: Adding 0 TAM to Kansas City
2023: Subtracting 0 TAM from Kansas City
2024: Adding 0 GAM to Kansas City
2024: Subtracting 0 GAM from Kansas City
2024: Adding 0 TAM to Kansas City
2024: Subtracting 0 TAM from Kansas City
2025: Adding 0 GAM to Kansas City
2025: Subtracting 0 

{'2018': {'New York City FC': {'GAM': 0, 'TAM': 0},
  'FC Cincinnati': {'GAM': -135000, 'TAM': -1050000},
  'New York Red Bulls': {'GAM': -225000, 'TAM': -150000},
  'Inter Miami': {'GAM': 0, 'TAM': 0},
  'Chicago': {'GAM': 1125000, 'TAM': -200000},
  'Charlotte FC': {'GAM': 0, 'TAM': 0},
  'Austin FC': {'GAM': 0, 'TAM': 0},
  'Houston': {'GAM': 0, 'TAM': -100000},
  'FC Dallas': {'GAM': 0, 'TAM': 225000},
  'CF Montreal': {'GAM': 0, 'TAM': -75000},
  'Portland': {'GAM': 0, 'TAM': 500000},
  'LAFC': {'GAM': 0, 'TAM': 0},
  'Atlanta United': {'GAM': 300000, 'TAM': 450000},
  'San Jose': {'GAM': 135000, 'TAM': 0},
  'Columbus Crew': {'GAM': 300000, 'TAM': 150000},
  'Vancouver Whitecaps': {'GAM': 225000, 'TAM': 200000},
  'Minnesota': {'GAM': 125000, 'TAM': 100000},
  'Seattle Sounders': {'GAM': 50000, 'TAM': 0},
  'Nashville': {'GAM': 0, 'TAM': 0},
  'D.C. United': {'GAM': -350000, 'TAM': -50000},
  'Colorado Rapids': {'GAM': -50000, 'TAM': 50000},
  'New England': {'GAM': 100000, 'TAM'

In [19]:
am_delta_table = pd.DataFrame({ 'year':[], 'club': [], 'gam_delta': [], 'tam_delta': []})
for year, data in am_table.items():
    for club, am in data.items():
        am_delta_table = am_delta_table.append({ 'year': year, 'club': club, 'gam_delta': am['GAM'], 'tam_delta': am['TAM'] }, ignore_index=True)
am_delta_table.to_csv('./allocation_money/mls_am_delta.csv', index=False)        
am_delta_table

,year,club,gam_delta,tam_delta
0,2018,New York City FC,0.0,0.0
1,2018,FC Cincinnati,-135000.0,-1050000.0
2,2018,New York Red Bulls,-225000.0,-150000.0
3,2018,Inter Miami,0.0,0.0
4,2018,Chicago,1125000.0,-200000.0
...,...,...,...,...
219,2025,Kansas City,0.0,0.0
220,2025,LA Galaxy,0.0,0.0
221,2025,Real Salt Lake,0.0,0.0
222,2025,Orlando City,0.0,0.0


In [20]:
# https://www.sounderatheart.com/2021/2/8/22273209/details-of-revised-mls-collective-bargaining-agreement
# https://www.mlssoccer.com/news/mls-announces-increase-targeted-allocation-money-2018-and-2019
# https://www.brotherlygame.com/2019/12/20/21031618/allocation-money-explained
base_gam = { 
    '2018' : (1200000 + 150000), # non-discretionary + league allocation
    '2019' : (1200000 + 150000), # non-discretionary + league allocation
    '2020' : 1525000, 
    '2021': 1525000,
    '2022' : 1625000,
    '2023' : 1900000,
    '2024' : 2585000,
    '2025' : 2930000,
    '2026' : 3280000,
    '2027' : 3921000
}

base_tam = { 
    '2018' : 2300000, # discretionary
    '2019' : 2400000, # discretionary
    '2020' : 2800000, 
    '2021': 2800000,
    '2022' : 2800000,
    '2023' : 2720000,
    '2024' : 2400000,
    '2025' : 2225000,
    '2026' : 2125000,
    '2027' : 2025000
}

am_delta_table['base_tam'] = am_delta_table.year.apply(lambda x: base_tam[x])
am_delta_table['base_gam'] = am_delta_table.year.apply(lambda x: base_gam[x])
am_delta_table.to_csv('./allocation_money/base_mls_am_budgets.csv', index=False)
am_delta_table

,year,club,gam_delta,tam_delta,base_tam,base_gam
0,2018,New York City FC,0.0,0.0,2300000,1350000
1,2018,FC Cincinnati,-135000.0,-1050000.0,2300000,1350000
2,2018,New York Red Bulls,-225000.0,-150000.0,2300000,1350000
3,2018,Inter Miami,0.0,0.0,2300000,1350000
4,2018,Chicago,1125000.0,-200000.0,2300000,1350000
...,...,...,...,...,...,...
219,2025,Kansas City,0.0,0.0,2225000,2930000
220,2025,LA Galaxy,0.0,0.0,2225000,2930000
221,2025,Real Salt Lake,0.0,0.0,2225000,2930000
222,2025,Orlando City,0.0,0.0,2225000,2930000


In [21]:
summary_alloc = adj_allocations.groupby(['asset_year', 'club'], as_index=False).gam_value.sum()

summary_alloc


,asset_year,club,gam_value
0,2018,CF Montreal,200000
1,2018,Colorado Rapids,400000
2,2018,Columbus Crew,50000
3,2018,D.C. United,200000
4,2018,FC Dallas,400000
...,...,...,...
92,2022,New England,200000
93,2022,New York City FC,250000
94,2022,San Jose,200000
95,2022,Seattle Sounders,200000


In [22]:
am_delta_table.year = am_delta_table.year.astype(int)

In [23]:
am_alloc_table = pd.merge(am_delta_table, summary_alloc, left_on=['year','club'], right_on=['asset_year','club'])
am_alloc_table

,year,club,gam_delta,tam_delta,base_tam,base_gam,asset_year,gam_value
0,2018,FC Dallas,0.0,225000.0,2300000,1350000,2018,400000
1,2018,CF Montreal,0.0,-75000.0,2300000,1350000,2018,200000
2,2018,LAFC,0.0,0.0,2300000,1350000,2018,1100000
3,2018,San Jose,135000.0,0.0,2300000,1350000,2018,50000
4,2018,Columbus Crew,300000.0,150000.0,2300000,1350000,2018,50000
...,...,...,...,...,...,...,...,...
92,2022,D.C. United,1975000.0,0.0,2800000,1625000,2022,250000
93,2022,Colorado Rapids,-400000.0,0.0,2800000,1625000,2022,200000
94,2022,New England,150000.0,0.0,2800000,1625000,2022,200000
95,2022,LA Galaxy,-500000.0,0.0,2800000,1625000,2022,200000


In [24]:
am_alloc_table['addtl_gam'] = am_alloc_table.gam_value
am_alloc_table.drop(['asset_year','gam_value'], axis=1, inplace=True)
am_alloc_table

,year,club,gam_delta,tam_delta,base_tam,base_gam,addtl_gam
0,2018,FC Dallas,0.0,225000.0,2300000,1350000,400000
1,2018,CF Montreal,0.0,-75000.0,2300000,1350000,200000
2,2018,LAFC,0.0,0.0,2300000,1350000,1100000
3,2018,San Jose,135000.0,0.0,2300000,1350000,50000
4,2018,Columbus Crew,300000.0,150000.0,2300000,1350000,50000
...,...,...,...,...,...,...,...
92,2022,D.C. United,1975000.0,0.0,2800000,1625000,250000
93,2022,Colorado Rapids,-400000.0,0.0,2800000,1625000,200000
94,2022,New England,150000.0,0.0,2800000,1625000,200000
95,2022,LA Galaxy,-500000.0,0.0,2800000,1625000,200000


In [25]:
am_alloc_table['available_tam'] = am_alloc_table['base_tam'] + am_alloc_table['tam_delta']
am_alloc_table['available_gam'] = am_alloc_table['base_gam'] + am_alloc_table['gam_delta'] + am_alloc_table['addtl_gam']
am_alloc_table['available_xam'] = am_alloc_table['available_gam'] + am_alloc_table['available_tam'] 
am_alloc_table


,year,club,gam_delta,tam_delta,base_tam,base_gam,addtl_gam,available_tam,available_gam,available_xam
0,2018,FC Dallas,0.0,225000.0,2300000,1350000,400000,2525000.0,1750000.0,4275000.0
1,2018,CF Montreal,0.0,-75000.0,2300000,1350000,200000,2225000.0,1550000.0,3775000.0
2,2018,LAFC,0.0,0.0,2300000,1350000,1100000,2300000.0,2450000.0,4750000.0
3,2018,San Jose,135000.0,0.0,2300000,1350000,50000,2300000.0,1535000.0,3835000.0
4,2018,Columbus Crew,300000.0,150000.0,2300000,1350000,50000,2450000.0,1700000.0,4150000.0
...,...,...,...,...,...,...,...,...,...,...
92,2022,D.C. United,1975000.0,0.0,2800000,1625000,250000,2800000.0,3850000.0,6650000.0
93,2022,Colorado Rapids,-400000.0,0.0,2800000,1625000,200000,2800000.0,1425000.0,4225000.0
94,2022,New England,150000.0,0.0,2800000,1625000,200000,2800000.0,1975000.0,4775000.0
95,2022,LA Galaxy,-500000.0,0.0,2800000,1625000,200000,2800000.0,1325000.0,4125000.0


In [26]:
am_alloc_table.to_csv('./allocation_money/mls_am_budgets.csv', index=False)

In [27]:
am_alloc_table[am_alloc_table.club == 'Atlanta United']

,year,club,gam_delta,tam_delta,base_tam,base_gam,addtl_gam,available_tam,available_gam,available_xam
22,2019,Atlanta United,0.0,750000.0,2400000,1350000,200000,3150000.0,1550000.0,4700000.0
43,2020,Atlanta United,0.0,750000.0,2800000,1525000,250000,3550000.0,1775000.0,5325000.0
65,2021,Atlanta United,375000.0,0.0,2800000,1525000,400000,2800000.0,2300000.0,5100000.0
88,2022,Atlanta United,300000.0,0.0,2800000,1625000,50000,2800000.0,1975000.0,4775000.0
